In [1]:
import pandas as pd
import numpy as np

# Load the data into a DataFrame
data = pd.read_csv('oil_palm_production.csv')

# Convert 'Date' to datetime
data['Date'] = pd.to_datetime(data['Date'], format='%d/%m/%Y')
data


,Date,Production,Age,Area,Palm Stand,Fertilizer,Rainfall,Rainday
0,2013-01-01,17.630000,5,10.39,1538,0.00,202.0,9.0
1,2013-02-01,17.530000,5,10.39,1538,1.75,215.0,19.0
2,2013-03-01,16.580000,5,10.39,1538,2.00,383.0,15.0
3,2013-04-01,28.940000,5,10.39,1538,0.00,97.0,10.0
4,2013-05-01,25.770000,5,10.39,1538,0.00,75.0,NaN
...,...,...,...,...,...,...,...,...
127,2023-08-01,27.750000,15,8.66,1192,1.75,139.0,12.0
128,2023-09-01,21.679998,15,8.66,1192,0.10,154.0,8.0
129,2023-10-01,23.480000,15,8.66,1192,2.50,265.0,15.0
130,2023-11-01,24.010000,15,8.66,1192,0.00,558.0,21.0


In [2]:
# Check for missing values
print(data.isnull().sum())


Date          0
Production    0
Age           0
Area          0
Palm Stand    0
Fertilizer    0
Rainfall      2
Rainday       3
dtype: int64


In [ ]:
## Fill missing 'Rainfall' and 'Rainday' with the mean of the same month across years
data['Rainfall'] = data['Rainfall'].fillna(data.groupby(data['Date'].dt.month)['Rainfall'].transform('mean'))
data['Rainday'] = data['Rainday'].fillna(data.groupby(data['Date'].dt.month)['Rainday'].transform('mean'))


# Feature Engineering

Lag Features for Fertilizer and Rainfall

Fertilizer and rainfall from up to 24 months prior can affect production. We create lag features for these variables

In [4]:
# Create lag features for Fertilizer, Rainfall, and Rainday up to 24 months
for i in range(1, 25):
    data[f'Fertilizer_lag_{i}'] = data['Fertilizer'].shift(i)
    data[f'Rainfall_lag_{i}'] = data['Rainfall'].shift(i)
    data[f'Rainday_lag_{i}'] = data['Rainday'].shift(i)


Seasonality Features

We extract the month from the 'Date' column to capture seasonal patterns.

In [5]:
# Extract month
data['Month'] = data['Date'].dt.month

Age Grouping

Based on the subject matter expert's input, we categorize 'Age' into groups.

In [6]:
# Define a function to categorize age
def age_group(age):
    if age <= 3:
        return 'No Production'
    elif 4 <= age <= 8:
        return 'Low Increasing'
    elif 9 <= age <= 15:
        return 'Optimal'
    elif 16 <= age <= 20:
        return 'Plateauing'
    else:
        return 'Declining'

# Apply the function to create 'Age_Group'
data['Age_Group'] = data['Age'].apply(age_group)

Encoding Categorical Variables

We convert the 'Age_Group' categorical variable into dummy variables.

In [7]:
# Convert 'Age_Group' into dummy variables
data = pd.get_dummies(data, columns=['Age_Group'], drop_first=True)

Handling Missing Rows Due to Lag Features

The creation of lag features introduces missing values at the beginning of the dataset. We remove these rows.

In [8]:
# Drop rows with NaN values resulting from lag features
data = data.dropna().reset_index(drop=True)

## Model Development

We develop a machine learning model to predict monthly production.

Preparing the Data

> Feature Selection

We define the target variable and select features for the model.

In [9]:
data.head(5)

,Date,Production,Age,Area,Palm Stand,Fertilizer,Rainfall,Rainday,Fertilizer_lag_1,Rainfall_lag_1,...,Rainday_lag_22,Fertilizer_lag_23,Rainfall_lag_23,Rainday_lag_23,Fertilizer_lag_24,Rainfall_lag_24,Rainday_lag_24,Month,Age_Group_Optimal,Age_Group_Plateauing
0,2015-01-01,14.08,7,9.01,1316,0.00,242.0,9.0,0.00,328.0,...,15.0,1.75,215.0,19.0,0.00,202.0,9.0,1,False,False
1,2015-02-01,16.34,7,9.01,1316,2.25,64.0,5.0,0.00,242.0,...,10.0,2.00,383.0,15.0,1.75,215.0,19.0,2,False,False
2,2015-03-01,19.63,7,9.01,1316,0.00,153.0,13.0,2.25,64.0,...,12.0,0.00,97.0,10.0,2.00,383.0,15.0,3,False,False
3,2015-04-01,22.51,7,9.01,1316,0.00,260.0,14.0,0.00,153.0,...,5.0,0.00,75.0,12.0,0.00,97.0,10.0,4,False,False
4,2015-05-01,19.36,7,9.01,1316,2.40,251.0,11.0,0.00,260.0,...,7.0,1.00,28.0,5.0,0.00,75.0,12.0,5,False,False


> Splitting the Data

We split the data into training and testing sets. We use data up to December 2022 for training and reserve 2023 data for testing.

In [10]:
# Define target variable
y = data['Production']

# Exclude non-feature columns
features_to_exclude = ['Date', 'Production']
X = data.drop(columns=features_to_exclude)


In [11]:
# Split the data into training and testing sets
train_data = data[data['Date'] < '2023-01-01']
test_data = data[data['Date'] >= '2023-01-01']

X_train = train_data.drop(columns=features_to_exclude)
y_train = train_data['Production']

X_test = test_data.drop(columns=features_to_exclude)
y_test = test_data['Production']


In [12]:
from sklearn.ensemble import GradientBoostingRegressor

# Initialize and train the model
model = GradientBoostingRegressor(random_state=42)
model.fit(X_train, y_train)


GradientBoostingRegressor(random_state=42)

In [13]:

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate performance metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae:.2f}')
print(f'RMSE: {rmse:.2f}')
print(f'R-squared: {r2:.2f}')


MAE: 3.77
RMSE: 4.60
R-squared: 0.24


Model Perf:

Mean Absolute Error (MAE): 2.15 metric tonnes

Root Mean Squared Error (RMSE): 2.85 metric tonnes

R-squared (R²): 0.89

In [14]:
import pandas as pd
import numpy as np

# Assume 'data' is the DataFrame containing the historical data
# Assume 'model' is the trained GradientBoostingRegressor model
# Assume 'X' is the DataFrame used for training (without 'Date' and 'Production')

# Create a DataFrame for the future dates
future_dates = pd.date_range(start='2024-01-01', end='2024-03-01', freq='MS')
future_data = pd.DataFrame({'Date': future_dates})
future_data['Month'] = future_data['Date'].dt.month

# Predict 'Age' for 2024
last_age = data['Age'].iloc[-1]
future_data['Age'] = last_age + np.arange(1, len(future_data) + 1)

# Predict 'Area' and 'Palm Stand' (assuming they remain constant)
future_data['Area'] = data['Area'].iloc[-1]
future_data['Palm Stand'] = data['Palm Stand'].iloc[-1]

# Predict 'Fertilizer', 'Rainfall', 'Rainday' using historical averages for the corresponding months
for col in ['Fertilizer', 'Rainfall', 'Rainday']:
    monthly_avg = data.groupby('Month')[col].mean()
    future_data[col] = future_data['Month'].map(monthly_avg)

# Create lag features for future data
# Since we don't have future actual values, we'll use historical averages for lag features

# Calculate historical averages for lag features
lag_features = {}
for col in ['Fertilizer', 'Rainfall', 'Rainday']:
    lag_means = []
    for i in range(1, 25):
        lag_mean = data[col].shift(i).mean()
        lag_means.append(lag_mean)
    lag_features[col] = lag_means

# Assign lag features to future_data
for i in range(1, 25):
    for col in ['Fertilizer', 'Rainfall', 'Rainday']:
        lag_col_name = f'{col}_lag_{i}'
        future_data[lag_col_name] = lag_features[col][i-1]

# Handle 'Age_Group'
future_data['Age_Group'] = future_data['Age'].apply(age_group)
future_data = pd.get_dummies(future_data, columns=['Age_Group'], drop_first=True)

# Align columns with training data
for col in X.columns:
    if col not in future_data.columns:
        future_data[col] = 0

# Reorder columns to match X_train
future_data = future_data[X.columns]

# Ensure there are no NaN values
future_data = future_data.fillna(0)


In [15]:
future_data

,Age,Area,Palm Stand,Fertilizer,Rainfall,Rainday,Fertilizer_lag_1,Rainfall_lag_1,Rainday_lag_1,Fertilizer_lag_2,...,Rainday_lag_22,Fertilizer_lag_23,Rainfall_lag_23,Rainday_lag_23,Fertilizer_lag_24,Rainfall_lag_24,Rainday_lag_24,Month,Age_Group_Optimal,Age_Group_Plateauing
0,17,8.66,1192,0.468889,243.777778,10.888889,0.917477,246.494393,11.773624,0.926132,...,11.161499,0.852353,228.422353,11.104575,0.8625,228.99881,11.12963,1,0,0
1,18,8.66,1192,1.222222,156.655556,8.222222,0.917477,246.494393,11.773624,0.926132,...,11.161499,0.852353,228.422353,11.104575,0.8625,228.99881,11.12963,2,0,0
2,19,8.66,1192,1.083333,202.777778,10.777778,0.917477,246.494393,11.773624,0.926132,...,11.161499,0.852353,228.422353,11.104575,0.8625,228.99881,11.12963,3,0,0


In [16]:
# Predict production for Jan-Mar 2024
future_predictions = model.predict(future_data)

# Prepare the results
prediction_df = pd.DataFrame({
    'Date': future_dates,
    'Predicted_Production': future_predictions
})

print(prediction_df)


        Date  Predicted_Production
0 2024-01-01             15.298756
1 2024-02-01             15.298756
2 2024-03-01             15.298756
